# 597 Group 5
Notes: most data cleaning work are done in other files, for this file, we take it for granted that all imported excel/csv files are correct.

In [1]:
import pandas as pd
import statsmodels.formula.api as sm
from patsy import dmatrices
from statsmodels.stats.outliers_influence import variance_inflation_factor

## Part 1 - Predit RENT_PER_SQFT by Multiple Factors on Zip Code Level

#### Data Cleaning

In [2]:
Age_Had_Birth = pd.read_excel("data/Age Had Birth.xlsx")
Age_Had_Birth['Zip Code'] = Age_Had_Birth['Zip Code'].astype(str)
Age_Had_Birth_sub = Age_Had_Birth[['Zip Code','35 to 50 years %']]
Age_Had_Birth_sub = Age_Had_Birth_sub.rename({'35 to 50 years %':'Had_Birth_35to50_pct'},axis=1)

In [3]:
Marital_Status = pd.read_excel("data/Marital Status.xlsx")
Marital_Status['Zip Code'] = Marital_Status['Zip Code'].astype(str)
Marital_Status['20 to 34 years % Never Married Males'] = Marital_Status['20 to 34 years % Never Married Males']/100
Marital_Status_sub = Marital_Status[['Zip Code','20 to 34 years % Never Married Males']]
Marital_Status_sub = Marital_Status_sub.rename({'20 to 34 years % Never Married Males':\
                                                'NeverMarriedMales_20to34_pct'},axis=1)

In [4]:
Earnings = pd.read_excel("data/Median Earnings.xlsx")
Earnings['Zip Code'] = Earnings['Zip Code'].astype(str).str.zfill(5)
Earnings['Median_Earnings'] = Earnings['Median_Earnings']/1000
Earnings = Earnings.rename({'Median_Earnings':'Median_Earnings_thousand'},axis=1)

In [5]:
English = pd.read_excel("data/English Speaking Households.xlsx")
English['Zip Code'] = English['Zip Code'].astype(str)
English = English.groupby('Zip Code')[['English_Speaking_Households_pct']].mean().reset_index()
English['English_Speaking_Households_pct'] = English['English_Speaking_Households_pct']/100

In [6]:
Migration = pd.read_excel("data/Migration.xlsx")
Migration['Zip Code'] = Migration['Zip Code'].astype(str)
Migration['Zip Code'] = Migration['Zip Code'].str.zfill(5)
Migration = Migration[Migration['MovedFromOtherStates_25YearsOver_pct']!='-']
Migration['MovedFromOtherStates_25YearsOver_pct'] = \
    Migration['MovedFromOtherStates_25YearsOver_pct'].astype(float)
Migration['MovedFromOtherStates_25YearsOver_pct'] = Migration['MovedFromOtherStates_25YearsOver_pct']/100

In [7]:
Computing_Device = pd.read_excel("data/Computing Device.xlsx")
Computing_Device['Zip Code'] = Computing_Device['Zip Code'].astype(str)
Computing_Device['Zip Code'] = Computing_Device['Zip Code'].str.zfill(5)
Computing_Device = Computing_Device[Computing_Device['Households_OneOrMore_Computer_pct']!='-']
Computing_Device['Households_OneOrMore_Computer_pct'] = Computing_Device['Households_OneOrMore_Computer_pct'].astype(float)
Computing_Device['Households_OneOrMore_Computer_pct'] = Computing_Device['Households_OneOrMore_Computer_pct']/100

In [8]:
Costco = pd.read_excel("data/Costco.xlsx")
Costco['Zip Code'] = Costco['Zip Code'].astype(str)
Costco_groupby = Costco.groupby('Zip Code')[['Store #']].count().reset_index()
Costco_groupby.rename({'Store #':'Num_Costco'},axis=1,inplace=True)

In [9]:
Walmart = pd.read_excel("data/Walmart.xlsx")
Walmart['Zip Code'] = Walmart['Zip Code'].astype(str)
Walmart_groupby = Walmart.groupby('Zip Code')[['Store #']].count().reset_index()
Walmart_groupby.rename({'Store #':'Num_Walmart'},axis=1,inplace=True)

In [10]:
Kroger = pd.read_excel("data/Kroger.xlsx")
Kroger['Zip Code'] = Kroger['Zip Code'].astype(str)
Kroger_groupby = Kroger.groupby('Zip Code')[['Store #']].count().reset_index()
Kroger_groupby.rename({'Store #':'Num_Kroger'},axis=1,inplace=True)

In [11]:
Starbucks = pd.read_excel("data/Starbucks.xlsx")
Starbucks = Starbucks[Starbucks['postalcode'].isnull()==False]
Starbucks['postalcode'] = Starbucks['postalcode'].astype(int).astype(str)
Starbucks['postalcode'] = Starbucks['postalcode'].str[:5]
Starbucks['postalcode'] = Starbucks['postalcode'].str.zfill(5)
Starbucks_groupby = Starbucks.groupby('postalcode')[['starbucksid']].count().reset_index()
Starbucks_groupby.rename({'postalcode':'Zip Code','starbucksid':'Num_Starbucks'},axis=1,inplace=True)

In [12]:
Museums = pd.read_csv("data/Museums.csv")
Museums_groupby = Museums.groupby('Zip Code (Administrative Location)')[['Museum ID']].count().reset_index()
Museums_groupby = Museums_groupby.rename({'Zip Code (Administrative Location)':'Zip Code','Museum ID':'Num_Museums'},axis=1)

/Users/siwenwang/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (9,14,21) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [13]:
Property = pd.read_excel('data/Property.xlsx',index_col=0)
Property['PROPERTY_ZIPCODE'] = Property['PROPERTY_ZIPCODE'].astype(str)
Rent_Per_Sqft = Property.groupby('PROPERTY_ZIPCODE')[['RENT_PER_SQFT']].median().reset_index()
Rent_Per_Sqft.rename({'PROPERTY_ZIPCODE':'Zip Code','RENT_PER_SQFT':'MED_RENT_PER_SQFT'},axis=1,inplace=True)
Rent_Per_Sqft['Zip Code'] = Rent_Per_Sqft['Zip Code'].astype(str)

#### Merge Variables and Perform Regression Analysis
Notes: we collected data on over 50 variables and have decided to build a regression model off the following 11 for better performance, the rest variables are deleted from this file.

In [14]:
Final = Rent_Per_Sqft.merge(Age_Had_Birth_sub, how='inner', on=['Zip Code'])
Final = Final.merge(Marital_Status_sub, how='inner', on=['Zip Code'])
Final = Final.merge(Earnings, how='inner', on=['Zip Code'])
Final = Final.merge(English, how='inner', on=['Zip Code'])
Final = Final.merge(Migration, how='inner', on=['Zip Code'])
Final = Final.merge(Computing_Device, how='inner', on=['Zip Code'])

Final = Final.merge(Costco_groupby, how='left', on=['Zip Code'])
Final.loc[Final['Num_Costco'].isnull(),'Num_Costco']=0
Final = Final.merge(Walmart_groupby, how='left', on=['Zip Code'])
Final.loc[Final['Num_Walmart'].isnull(),'Num_Walmart']=0
Final = Final.merge(Kroger_groupby, how='left', on=['Zip Code'])
Final.loc[Final['Num_Kroger'].isnull(),'Num_Kroger']=0
Final = Final.merge(Starbucks_groupby, how='left', on=['Zip Code'])
Final.loc[Final['Num_Starbucks'].isnull(),'Num_Starbucks']=0
Final = Final.merge(Museums_groupby, how='left', on=['Zip Code'])
Final.loc[Final['Num_Museums'].isnull(),'Num_Museums']=0

In [15]:
variables1 = "MED_RENT_PER_SQFT ~ Had_Birth_35to50_pct + NeverMarriedMales_20to34_pct  + \
    Median_Earnings_thousand + English_Speaking_Households_pct +\
    MovedFromOtherStates_25YearsOver_pct + Households_OneOrMore_Computer_pct +\
    Num_Costco + Num_Walmart + Num_Kroger + Num_Starbucks + Num_Museums"

In [16]:
q1 = sm.ols(formula=variables1,data=Final).fit()
print(q1.summary())

                            OLS Regression Results                            
Dep. Variable:      MED_RENT_PER_SQFT   R-squared:                       0.474
Model:                            OLS   Adj. R-squared:                  0.472
Method:                 Least Squares   F-statistic:                     217.5
Date:                Mon, 21 Mar 2022   Prob (F-statistic):               0.00
Time:                        12:38:14   Log-Likelihood:                -724.78
No. Observations:                2662   AIC:                             1474.
Df Residuals:                    2650   BIC:                             1544.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                                           coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------


### For each Zip Code:
- **MED_RENT_PER_SQFT**: median rent per sqft


- **Had_Birth_35to50_pct**: what percentage of females who had birth in 2019 are between age 35 and 50 years\
Census Bereau: ACSDT5Y2019.B13002_metadata_2022-03-06T181557


- **NeverMarriedMales_20to34_pct**：what percentage of males with age from 20 to 34 years are never married\
Census Bereau: ACSST5Y2019.S1201_metadata_2021-12-08T054544


- **Median_Earnings_thousand**: median household income


- **English_Speaking_Households_pct**：what percenatage of households are English speaking households


- **MovedFromOtherStates_25YearsOver_pct**： what percentage of people with age 25 years and over are moved from other states


- **Households_OneOrMore_Computer_pct**: what percentage of households possess one or more desktops or laptops


- **Num_Costco**: number of Costcos in it


- **Num_Walmart**: number of Walmarts in it


- **Num_Kroger**: number of Krogers in it


- **Num_Starbucks**: number of Starbucks in it


- **Num_Museums**: number of museums, aquariums, zoos, etc. in it


In [17]:
# VIF
y1, X1 = dmatrices(variables1, Final, return_type='dataframe')
vif1 = pd.DataFrame()
vif1["VIF Factor"] = [variance_inflation_factor(X1.values, i) for i in range(X1.shape[1])]
vif1["features"] = X1.columns
vif1.round(1)

,VIF Factor,features
0,132.7,Intercept
1,1.2,Had_Birth_35to50_pct
2,1.2,NeverMarriedMales_20to34_pct
3,2.1,Median_Earnings_thousand
4,1.3,English_Speaking_Households_pct
5,1.2,MovedFromOtherStates_25YearsOver_pct
6,2.2,Households_OneOrMore_Computer_pct
7,1.1,Num_Costco
8,1.1,Num_Walmart
9,1.1,Num_Kroger


#### Predit RENT_PER_SQFT Using the Model, Compare it to the Actual Value

In [18]:
ypred1 = q1.predict()
print(ypred1)

[1.49771679 1.53820903 1.28525166 ... 1.55648495 1.41477296 1.30380633]


In [19]:
Comparison1 = Final.loc[:,['Zip Code','MED_RENT_PER_SQFT']]
Comparison1.loc[:,'Zip_Prediction'] = ypred1
Comparison1.loc[:,'Zip_Diff'] = ypred1 - Final['MED_RENT_PER_SQFT']
Comparison1.loc[:,'Zip_Rank'] = Comparison1['Zip_Diff'].rank(ascending=False).astype(int)
Comparison1 = Comparison1.sort_values(by='Zip_Diff',ascending=False)

#### The following shows the top 5 Zip Codes with the highest potential for rent raising

In [20]:
Comparison1.head(5)

,Zip Code,MED_RENT_PER_SQFT,Zip_Prediction,Zip_Diff,Zip_Rank
2572,87116,0.926792,2.140326,1.213533,1
2612,89109,1.347579,2.544902,1.197323,2
462,31409,1.489480,2.469451,0.979971,3
2174,79849,1.100953,2.073549,0.972596,4
2338,83422,0.857306,1.823393,0.966087,5


## Part 2 - Predict RENT_PER_SQFT by Property Ranking & Age

#### Data Cleaning

In [21]:
Property = Property[Property['PROPERTY_IMPRATING'].isnull() == False]
Property = Property[Property['PROPERTY_LOCRATING'].isnull() == False]
Property.loc[Property['PROPERTY_IMPRATING']=='B ','PROPERTY_IMPRATING']='B'
Property.loc[Property['PROPERTY_IMPRATING']=='C ','PROPERTY_IMPRATING']='C'
Property.loc[Property['PROPERTY_LOCRATING']=='B ','PROPERTY_LOCRATING']='B'
Property.loc[Property['PROPERTY_LOCRATING']=='C ','PROPERTY_LOCRATING']='C'

#### Regression Analysis

In [22]:
variables2 = "RENT_PER_SQFT ~ PROPERTY_IMPRATING + PROPERTY_LOCRATING + PROPERTY_YEAR"

In [23]:
q2 = sm.ols(formula=variables2,data=Property).fit()
print(q2.summary())

                            OLS Regression Results                            
Dep. Variable:          RENT_PER_SQFT   R-squared:                       0.291
Model:                            OLS   Adj. R-squared:                  0.291
Method:                 Least Squares   F-statistic:                     573.5
Date:                Mon, 21 Mar 2022   Prob (F-statistic):               0.00
Time:                        12:38:15   Log-Likelihood:                -14234.
No. Observations:               26530   AIC:                         2.851e+04
Df Residuals:                   26510   BIC:                         2.867e+04
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept               

In [24]:
y2, X2 = dmatrices(variables2, Property, return_type='dataframe')
vif2 = pd.DataFrame()
vif2["VIF Factor"] = [variance_inflation_factor(X2.values, i) for i in range(X2.shape[1])]
vif2["features"] = X2.columns
vif2.round(1)

,VIF Factor,features
0,34.0,Intercept
1,1.0,PROPERTY_IMPRATING[T.A+]
2,2.1,PROPERTY_IMPRATING[T.A-]
3,2.7,PROPERTY_IMPRATING[T.B]
4,2.0,PROPERTY_IMPRATING[T.B+]
5,3.2,PROPERTY_IMPRATING[T.B-]
6,3.1,PROPERTY_IMPRATING[T.C]
7,4.0,PROPERTY_IMPRATING[T.C+]
8,1.4,PROPERTY_IMPRATING[T.C-]
9,1.0,PROPERTY_IMPRATING[T.D]


#### Predit RENT_PER_SQFT Using the Model, Compare it to the Actual Value

In [25]:
ypred2 = q2.predict()
print(ypred2)

[1.39523574 1.17829843 1.31972071 ... 1.70511282 1.85207903 1.64291895]


In [26]:
Comparison2 = Property.loc[:,['PROPERTY_ID','PROPERTY_ZIPCODE','PROPERTY_SQFT','RENT_PER_SQFT']]
Comparison2.rename({'PROPERTY_ZIPCODE':'Zip Code'},axis=1,inplace=True)
Comparison2.loc[:,'Prop_Prediction'] = ypred2
Comparison2.loc[:,'Prop_Diff'] = ypred2 - Property['RENT_PER_SQFT']
Comparison2.loc[:,'Prop_Rank'] = Comparison2['Prop_Diff'].rank(ascending=False).astype(int)
Comparison2 = Comparison2.sort_values(by='Prop_Rank')

#### The following shows the top 5 properties with the highest potential for rent raising (solely according to the above model)

In [27]:
Comparison2.head(5)

,PROPERTY_ID,Zip Code,PROPERTY_SQFT,RENT_PER_SQFT,Prop_Prediction,Prop_Diff,Prop_Rank
10883,61_1508,78258,480460,1.295508,2.451608,1.156099,1
5723,124_97,30809,260404,1.087664,2.196034,1.108370,2
538,14_6466,89149,112404,1.162734,2.260548,1.097814,3
4794,54_188,72211,571835,1.392445,2.431280,1.038834,4
1330,104_107,37803,53450,0.564079,1.584505,1.020426,5


## Part 3 - Demand: Population Growth and Unemployment Rate Growth by Zip

In [28]:
Population_Growth = pd.read_excel('data/Population Growth.xlsx',index_col=0)
Population_Growth['Zip Code'] = Population_Growth['Zip Code'].astype(int).astype(str)
Population_Growth['Zip Code'] = Population_Growth['Zip Code'].str.zfill(5)
Population_Growth['Population_Rank'] = Population_Growth['Population_Avg_Yearly_Growth'].\
                                    rank(ascending=False).astype(int)

In [29]:
Unemployment_Rate_Growth = pd.read_excel('data/Unemployment Rate Growth.xlsx',index_col=0)
Unemployment_Rate_Growth['Zip Code'] = Unemployment_Rate_Growth['Zip Code'].astype(int).astype(str)
Unemployment_Rate_Growth['Zip Code'] = Unemployment_Rate_Growth['Zip Code'].str.zfill(5)
Unemployment_Rate_Growth['Unemployment_Rank'] = Unemployment_Rate_Growth\
                                            ['UnemploymentRate_Avg_Yearly_Growth'].rank().astype(int)

## Final Part - Merge all three parts to select ideal properties for investment

#### Merge all three parts on property level, now for each property, we can see the following metrics:
- **PROPERTY_ID, Zip Code, PROPERTY_SQFT, RENT_PER_SQFT**: self-explanatory

- **Prof_Prediction**: the predicted rent_per_sqft using model from part 2
- **Prop_Diff**: Prof_Prediction - RENT_PER_SQFT
- **Prop_Rank**: the rank of Prop_Diff among all properties, the bigger the difference, the better the rank

- **MED_RENT_PER_SQFT**: the actual median rent_per_sqft of a zipcode
- **Zip_Prediction**: the predicted rent_per_sqft using model from part 1
- **Zip_Diff**: Zip_Prediction - MED_RENT_PER_SQFT 
- **Zip_Rank**: the rank of Zip_Diff among all Zip Codes, the bigger the difference, the better the rank
- **Population_Avg_Yearly_Growth**: from 2016 to 2020, the population avg yearly growth of a Zip Code
- **Population_Rank**: the rank of Population_Avg_Yearly_Growth among all Zip Codes, the larger the growth, the better the rank
- **UnemploymentRate_Avg_Yearly_Growth** from 2016 to 2020, the unemployment rate avg yearly growth of a Zip Code
- **Unemployment_Rank**: the rank of UnemploymentRate_Avg_Yearly_Growth among all Zip Codes, the smaller the growth, the better the rank
- **Dollar**: the estimated dollars gain from rent raising

In [30]:
merge_Comparison = Comparison2.merge(Comparison1,how='inner',on='Zip Code')
merge_Comparison = merge_Comparison.merge(Population_Growth,how='inner',on='Zip Code')
merge_Comparison = merge_Comparison.merge(Unemployment_Rate_Growth,how='inner',on='Zip Code')
merge_Comparison['Dollar'] = (((merge_Comparison['Zip_Prediction']+merge_Comparison['Prop_Prediction'])/2)\
                                    -merge_Comparison['RENT_PER_SQFT'])*merge_Comparison['PROPERTY_SQFT']
merge_Comparison

,PROPERTY_ID,Zip Code,PROPERTY_SQFT,RENT_PER_SQFT,Prop_Prediction,Prop_Diff,Prop_Rank,MED_RENT_PER_SQFT,Zip_Prediction,Zip_Diff,Zip_Rank,Population_Avg_Yearly_Growth,Population_Rank,UnemploymentRate_Avg_Yearly_Growth,Unemployment_Rank,Dollar
0,61_1508,78258,480460,1.295508,2.451608,1.156099,1,1.448756,1.941987,0.493232,105,0.008042,13814,-0.057738,13066,433033.368249
1,61_1183,78258,339410,1.351463,2.139467,0.788004,139,1.448756,1.941987,0.493232,105,0.008042,13814,-0.057738,13066,233943.221482
2,61_1631,78258,151448,1.198986,1.907966,0.708981,336,1.448756,1.941987,0.493232,105,0.008042,13814,-0.057738,13066,109949.908852
3,61_1936,78258,355536,1.292021,1.995960,0.703939,350,1.448756,1.941987,0.493232,105,0.008042,13814,-0.057738,13066,240681.037389
4,61_1277,78258,265834,1.289338,1.945678,0.656339,573,1.448756,1.941987,0.493232,105,0.008042,13814,-0.057738,13066,173986.822977
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26304,167_933329,83001,43759,4.417148,1.222295,-3.194853,26527,3.521454,2.044872,-1.476582,2660,0.023221,8440,-0.030298,15423,-121806.009380
26305,56_474,33480,78355,2.963053,1.445771,-1.517282,26410,4.317562,1.942387,-2.375175,2662,0.019693,9385,0.002979,17927,-99430.444998
26306,56_61,33480,122600,5.672072,2.167812,-3.504260,26530,4.317562,1.942387,-2.375175,2662,0.019693,9385,0.002979,17927,-443440.826290
26307,107_616,29401,128129,3.349187,1.648088,-1.701099,26459,3.881589,2.200769,-1.680820,2661,-0.025138,26770,-0.133046,6350,-182552.913186


#### Rule 1: All recommanded properties have to be in Zip Code with positive population growth and negative unemployment rate growth

In [31]:
merge_Comparison = merge_Comparison[merge_Comparison['Population_Avg_Yearly_Growth']>0]
merge_Comparison = merge_Comparison[merge_Comparison['UnemploymentRate_Avg_Yearly_Growth']<0]

#### Rule 2: All recommanded properties have to have the potential for rent raising according to the predictions from part 1 and 2

In [32]:
merge_Comparison = merge_Comparison[merge_Comparison['Zip_Diff']>0]
merge_Comparison = merge_Comparison[merge_Comparison['Prop_Diff']>0]

#### Method 1: Adjust threshold until 5 properties are left

In [33]:
merge_Comparison1 = merge_Comparison[merge_Comparison['Zip_Rank']<30]
merge_Comparison1 = merge_Comparison1[merge_Comparison1['Prop_Rank']<163]
# merge_Comparison1 = merge_Comparison1[merge_Comparison1['Population_Avg_Yearly_Growth']>0]
# merge_Comparison1 = merge_Comparison1[merge_Comparison1['UnemploymentRate_Avg_Yearly_Growth']<0]

In [34]:
merge_Comparison1.shape

(5, 16)

In [35]:
merge_Comparison1

,PROPERTY_ID,Zip Code,PROPERTY_SQFT,RENT_PER_SQFT,Prop_Prediction,Prop_Diff,Prop_Rank,MED_RENT_PER_SQFT,Zip_Prediction,Zip_Diff,Zip_Rank,Population_Avg_Yearly_Growth,Population_Rank,UnemploymentRate_Avg_Yearly_Growth,Unemployment_Rank,Dollar
382,118_89,39194,77600,0.564433,1.446010,0.881577,29,0.564433,1.267295,0.702862,21,0.016781,10289,-0.051085,13623,61476.236345
989,73_361,35406,283248,1.129470,1.944433,0.814963,102,1.009356,1.711944,0.702588,22,0.024065,8237,-0.052988,13473,197910.693556
1260,54_140,72207,62900,0.660095,1.446888,0.786792,143,0.861210,1.571249,0.710039,20,0.009459,13147,-0.162930,4514,53400.381846
1316,89_451,79927,144436,0.744025,1.522952,0.778927,155,0.744025,1.673623,0.929598,7,0.011850,12110,-0.134814,6218,123386.282647
1344,119_1088,36527,202288,1.189433,1.967263,0.777831,162,1.207160,1.872163,0.665003,27,0.034213,6258,-0.006602,17276,147726.931134


In [36]:
merge_Comparison1['PROPERTY_ID']

382       118_89
989       73_361
1260      54_140
1316      89_451
1344    119_1088
Name: PROPERTY_ID, dtype: object

#### Method 2: Set thresholds for Zip_Rank and Prop_Rank to be top 10%, select  5 properties with highest 'Dollar'

In [37]:
len(merge_Comparison['Zip Code'].unique())*0.1

74.60000000000001

In [38]:
len(merge_Comparison['PROPERTY_ID'].unique())*0.1

574.0

In [39]:
merge_Comparison2 = merge_Comparison[merge_Comparison['Zip_Rank']<74]
merge_Comparison2 = merge_Comparison2[merge_Comparison2['Prop_Rank']<574]

In [40]:
merge_Comparison2.shape

(26, 16)

In [41]:
merge_Comparison2 = merge_Comparison2.sort_values(by='Dollar',ascending=False).head(5)
merge_Comparison2

,PROPERTY_ID,Zip Code,PROPERTY_SQFT,RENT_PER_SQFT,Prop_Prediction,Prop_Diff,Prop_Rank,MED_RENT_PER_SQFT,Zip_Prediction,Zip_Diff,Zip_Rank,Population_Avg_Yearly_Growth,Population_Rank,UnemploymentRate_Avg_Yearly_Growth,Unemployment_Rank,Dollar
1350,178_1110800,72712,520200,1.024221,1.755395,0.731173,269,1.011632,1.668318,0.656686,29,0.050725,4387,-0.064867,12356,357707.714324
1969,94_166,74075,439359,0.829133,1.564846,0.735713,256,0.845600,1.477676,0.632075,35,0.022032,8739,-0.088571,10093,304092.540451
1581,46_1554,27519,437947,1.664379,2.431280,0.766900,185,1.504259,2.198174,0.693915,24,0.038429,5679,-0.032516,15244,284817.710109
1972,94_87,74075,423128,0.849984,1.557784,0.707800,339,0.845600,1.477676,0.632075,35,0.022032,8739,-0.088571,10093,282542.017736
1349,178_1110703,72712,358576,0.952970,1.698051,0.745082,232,1.011632,1.668318,0.656686,29,0.050725,4387,-0.064867,12356,261837.641540


In [42]:
merge_Comparison2['PROPERTY_ID']

1350    178_1110800
1969         94_166
1581        46_1554
1972          94_87
1349    178_1110703
Name: PROPERTY_ID, dtype: object